# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
#pip install --upgrade langchain

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
#pip install docarray

In [6]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [7]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [8]:
response = index.query(query)

In [9]:
display(Markdown(response))



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated sun protection, 100% polyester fabric, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated sun protection, 52% polyester and 48% nylon fabric, wrinkle-free, quickly evaporates perspiration, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated sun protection, 78% nylon and 22% Lycra Xtra Life fiber fabric, wicks moisture for quick-drying comfort, abrasion resistant, fits comfortably over swimsuit |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated sun protection, 71% Nylon and 29% Polyester fabric, wrinkle resistant, front and back cape venting, two front bellows pockets |

All four shirts provide UPF 50+ rated sun protection, blocking 98% of the sun's harmful rays. The fabrics used vary from 100% polyester, 52% polyester and

## Step By Step

In [10]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [11]:
docs = loader.load()

In [12]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT?? antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [13]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [14]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [15]:
print(len(embed))

1536


In [16]:
print(embed[:5])

[-0.021867522969841957, 0.006806864403188229, -0.01818099617958069, -0.03910486772656441, -0.014066680334508419]


In [17]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [18]:
query = "Please suggest a shirt with sunblocking"

In [19]:
docs = db.similarity_search(query)

In [20]:
len(docs)

4

In [21]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun ??? our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated ??? the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [22]:
retriever = db.as_retriever()

In [23]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [24]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [25]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [26]:
display(Markdown(response))

| Shirt Name | Description |
| --- | --- |
| Sun Shield Shirt | A high-performance sun shirt that provides UPF 50+ sun protection, blocks 98% of harmful UV rays, and is made of 78% nylon and 22% Lycra Xtra Life fiber. It wicks moisture for quick-drying comfort, fits comfortably over swimsuits, and is abrasion-resistant. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Originally designed for fishing, this shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. |
| Men's TropicVibe Shirt, Short-Sleeve | This shirt has built-in UPF 50+ sun protection, a lightweight feel, and a traditional fit that is relaxed through the chest, sleeve, and waist. Made of 71% nylon and 29% polyester, it is wrinkle-resistant and has front and back cape venting for cool breezes. |
| Men's Tropical Plaid Short-Sleeve Shirt | This shirt is rated UPF 50+ for superior protection from the sun's UV rays. With a traditional fit that is relaxed through the chest, sleeve, and waist, it is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting and two front bellows pockets. |

All of these shirts provide high levels of sun protection, with UPF 50+ ratings and the ability to block 98% of harmful UV rays. They are made of high-performance fabrics that are quick-drying, wrinkle-resistant, and comfortable to wear. The Sun Shield Shirt is abrasion-resistant, while the Men's Plaid Tropic Shirt is designed for fishing and travel. The Men's TropicVibe Shirt has front and back cape venting for cool breezes, and the Men's Tropical Plaid Short-Sleeve Shirt has two front bellows pockets.

In [30]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [31]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [32]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [33]:
display(Markdown(response))

| Shirt Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. Provides the highest rated sun protection possible. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Rated to UPF 50+, helping you stay cool and dry. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available. |
| Sun Shield Shirt by | High-performance sun shirt is guaranteed to protect from harmful UV rays. Made of 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated. Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion-resistant for season after season of wear. |
| Men's TropicVibe Shirt, Short-Sleeve | Built-in UPF 50+ has the lightweight feel you want and the coverage you need when the air is hot and the UV rays are strong. Made of 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. Wrinkle-resistant. Front and back cape venting lets in cool breezes. Two front bellows pockets. Provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. | 

The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. It is rated UPF 50+ for superior protection from the sun's UV rays. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon and is machine washable and dryable. It is rated to UPF 50+ and has front and back cape venting, two front bellows pockets, and an imported design. The Sun Shield Shirt by is made of 78% nylon, 22% Lycra Xtra Life fiber and is UPF 50+ rated. It wicks moisture for quick-drying comfort, fits comfortably over your favorite swimsuit, and is abrasion-resistant. The Men's TropicVibe Shirt, Short-Sleeve has built-in UPF 50+ and is made of 71% Nylon, 29% Polyester. It is wrinkle-resistant, has front and back cape venting, two front bellows pockets, and provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays.

In [34]:
response = index.query(query, llm=llm)

In [35]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.